# google_schedule_to_gridy

グーグルカレンダーの予定をgridyに自動でぶち込みたい

どうやらAPIがあってPythonから触れるらしい。後々バンドリのカレンダーの方にも実装しとこう

APIの設定とかの話はこっち

https://blowup-bbs.com/google-calender-api-python-quickstart/

APIで作製したJsonファイルは実行ファイルと同じ階層においておく

ソースとかの話はこっち

https://non-dimension.com/python-googlecalendarapi/#toc9

ここからはquickstart.pyの中身についてみていく

In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [2]:
# これらのスコープを変更する場合は、token.pickleというファイルを削除してください。
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

Google Calendar APIの基本的な使用方法を示します。

ユーザーのカレンダーにある次の10個のイベントの開始と名前を表示します。

In [3]:
creds = None
# token.pickleファイルには、ユーザーのアクセストークンと
# リフレッシュトークンが格納されており、認証フローが初めて完了したときに
# 自動的に作成されます。
if os.path.exists('../token.pickle'):
    # gitにupする関係上ログイン関係ファイルは1階層下に配置している
    with open('../token.pickle', 'rb') as token:
        creds = pickle.load(token)

In [4]:
# 利用可能な（有効な）認証情報がない場合は、ユーザーのログインを許可します。
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('../token.pickle', 'wb') as token:
        pickle.dump(creds, token)

The authentication flow has completed. You may close this window.（認証フローが完了しました。このウィンドウを閉じても構いません。）が出たらOK

In [5]:
# たぶん、カレンダーだよって主張している
service = build('calendar', 'v3', credentials=creds)

In [6]:
# Calendar APIの呼び出し
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

Getting the upcoming 10 events


- calendarId: string, カレンダーの識別子。カレンダー ID を取得するには、calendarList.list メソッドを呼び出します。現在ログインしているユーザのプライマリ・カレンダーにアクセスしたい場合は、"primary" キーワードを使用します。(必須)
    - IDの確認方法はGoogleカレンダーの【設定】>【マイカレンダーの設定】から確認したいカレンダーを選択し、【カレンダーの統合】で【カレンダー ID】が確認できます。
- timeMin: 予定を取得する最小時刻
- maxResults: APIが返す最大の予定数
- singleEvents: boolean, 繰り返し発生するイベントをインスタンスに展開し、単一の単発イベントと繰り返し発生するイベントのインスタンスのみを返し、基本となる繰り返し発生するイベント自体は返さないかどうか。オプションです。デフォルトはFalseです。
    - 日本語ガバガバでどっちやねん
- orderBy: string, 結果として返されるイベントの順序。任意です。デフォルトは、指定されていない安定した順序です。

In [7]:
print(type(events))
print(type(events[0]))
print(events[0].keys())

<class 'list'>
<class 'dict'>
dict_keys(['kind', 'etag', 'id', 'status', 'htmlLink', 'created', 'updated', 'summary', 'description', 'creator', 'organizer', 'start', 'end', 'iCalUID', 'sequence', 'reminders', 'eventType'])


色々あるけど、代表的な（自分が使う）奴をメモ
- 'summary': 予定名
- 'start': {'date': '2021-04-10'}
- 'end': {'date': '2021-04-11'}